In [ ]:
from IPython.display import display
import serialhub
W = serialhub.SerialHubWidget();
display(W)
W.serial_options = { 'baudRate': 115200 }

In [ ]:
W.value = "CLEARED"
s = f"Test {W.status}"
W.write_str(s)
s